In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI 迁移：AutoML 图像目标检测

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/sdk-automl-image-object-detection-batch-online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/sdk-automl-image-object-detection-batch-online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/migration/sdk-automl-image-object-detection-batch-online.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI 工作台中打开
    </a>
  </td>                                                                                               
</table>

<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex AI SDK来训练和部署AutoML目标检测模型。

了解更多关于[迁移到Vertex AI](https://cloud.google.com/vertex-ai/docs/start/migrating-to-vertex-ai)和[图像数据的目标检测](https://cloud.google.com/vertex-ai/docs/training-overview#object_detection_for_images)。

### 目标

在本教程中，您将学习如何使用 `AutoML` 训练图像模型，并使用 `Vertex AI Prediction` 和 `Vertex AI Batch Prediction` 进行在线和批量预测。

本教程使用以下 Google Cloud ML 服务：

- `AutoML`
- `Vertex AI Batch Prediction`
- `Vertex AI Model` 资源
- `Vertex AI Endpoint` 资源

执行的步骤包括：

- 训练一个 AutoML 目标检测模型。
- 进行批量预测。
- 将模型部署到端点。
- 进行在线预测。

数据集

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/open_images_v4)中的沙拉类别的[OpenImages数据集](https://www.tensorflow.org/datasets/catalog/open_images_v4)。这个数据集不需要任何特征工程。在本教程中使用的数据集版本存储在一个公共云存储桶中。训练好的模型可以预测图像中沙拉类别的边界框位置和相应类型，包括五种项目：沙拉、海鲜、番茄、烘焙食品或奶酪。

### 成本

本教程使用了Google Cloud 的收费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI
价格](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage
价格](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)
根据您的预期使用量生成成本估算。

## 安装

安装执行此笔记本所需的软件包。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-storage \
                                 tensorflow

仅限Colab使用：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

在你开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。 了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

UUID

如果您在一个实时教程会话中，可能正在使用一个共享的测试帐户或项目。为了避免用户在创建资源时发生名称冲突，您可以为每个实例会话创建一个UUID，并将其附加到您在本教程中创建的资源名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的 Google Cloud 账户

根据您的 Jupyter 环境，您可能需要手动进行身份验证。请按照以下相关说明操作。

**1. Vertex AI Workbench**
* 无需操作，因为您已经通过验证。

**2. 本地 JupyterLab 实例，请取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账户或其他
* 查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples为您的服务账户授予云存储权限。

创建一个云存储桶

创建一个存储桶用于存储中间产物，例如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有你的存储桶尚未存在时：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l {REGION} {BUCKET_URI}

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

初始化Python的Vertex AI SDK

为您的项目和相应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

云存储培训数据的位置。

现在将变量 `IMPORT_FILE` 设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-samples-data/vision/salads.csv"

快速查看您的数据

此教程使用存储在公共云存储桶中的Salads数据集的一个版本，使用CSV索引文件。

首先快速查看数据。您可以通过计算CSV索引文件中的行数（`wc -l`）来统计示例数量，然后查看前几行。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

创建数据集

### [datasets.create-dataset-api](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api)

### [datasets.create-dataset-api](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api)

### 创建数据集

接下来，使用`ImageDataset`类的`create`方法创建`Dataset`资源，该方法接受以下参数：

- `display_name`：`Dataset`资源的可读名称。
- `gcs_source`：包含一个或多个数据集索引文件，用于将数据项导入到`Dataset`资源中。
- `import_schema_uri`：数据项的数据标记模式。

此操作可能需要几分钟时间。

In [ ]:
dataset = aip.ImageDataset.create(
    display_name="Salads" + "_" + UUID,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aip.schema.dataset.ioformat.image.bounding_box,
)

print(dataset.resource_name)

INFO:google.cloud.aiplatform.datasets.dataset:创建ImageDataset
INFO:google.cloud.aiplatform.datasets.dataset:创建ImageDataset支持的LRO: projects/759209241365/locations/us-central1/datasets/2940964905882222592/operations/1941426647739662336
INFO:google.cloud.aiplatform.datasets.dataset:已创建ImageDataset。资源名称: projects/759209241365/locations/us-central1/datasets/2940964905882222592
INFO:google.cloud.aiplatform.datasets.dataset:在另一个会话中使用此ImageDataset：
INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.ImageDataset('projects/759209241365/locations/us-central1/datasets/2940964905882222592')
INFO:google.cloud.aiplatform.datasets.dataset:导入ImageDataset数据: projects/759209241365/locations/us-central1/datasets/2940964905882222592
INFO:google.cloud.aiplatform.datasets.dataset:导入ImageDataset数据支持的LRO: projects/759209241365/locations/us-central1/datasets/2940964905882222592/operations/8100099138168815616
INFO:google.cloud.aiplatform.datasets.dataset:ImageDataset数据已导入。资源名称: projects/759209241365/locations/us-central1/datasets/2940964905882222592
projects/759209241365/locations/us-central1/datasets/2940964905882222592

训练一个模型

### [training.automl-api](https://cloud.google.com/vertex-ai/docs/training/automl-api)

### [训练自动机 API](https://cloud.google.com/vertex-ai/docs/training/automl-api)

### 创建并运行训练流水线

要训练一个AutoML模型，您需要执行两个步骤：1) 创建一个训练流水线，2) 运行这个流水线。

#### 创建训练流水线

使用`AutoMLImageTrainingJob`类创建一个AutoML训练流水线，需要以下参数：

- `display_name`：`TrainingJob`资源的可读名称。
- `prediction_type`：训练模型的任务类型。
  - `classification`：图像分类模型。
  - `object_detection`：图像目标检测模型。
- `multi_label`：如果是分类任务，是单标签（`False`）还是多标签（`True`）。
- `model_type`：部署模型的类型。
  - `CLOUD`：部署在Google Cloud上
  - `CLOUD_HIGH_ACCURACY_1`：为在Google Cloud上部署优化准确性而非延迟的模型。
  - `CLOUD_LOW_LATENCY_`：为在Google Cloud上部署优化延迟而非准确性的模型。
  - `MOBILE_TF_VERSATILE_1`：部署在边缘设备上。
  - `MOBILE_TF_HIGH_ACCURACY_1`：为在边缘设备上部署优化准确性而非延迟的模型。
  - `MOBILE_TF_LOW_LATENCY_1`：为在边缘设备上部署优化延迟而非准确性的模型。
- `base_model`：（可选）从现有`Model`资源进行迁移学习，仅支持图像分类。

实例化的对象是用于训练任务的有向无环图（DAG）。

In [ ]:
dag = aip.AutoMLImageTrainingJob(
    display_name="salads_" + UUID,
    prediction_type="object_detection",
    multi_label=False,
    model_type="CLOUD",
    base_model=None,
)

print(dag)

*示例输出：*

<google.cloud.aiplatform.training_jobs.AutoMLImageTrainingJob对象，位于0x7f806a6116d0>

#### 运行训练管道

接下来，您可以通过调用`run`方法并传入以下参数来运行DAG以开始训练作业：

- `dataset`: 用于训练模型的`Dataset`资源。
- `model_display_name`: 训练模型的人类可读名称。
- `training_fraction_split`: 用于训练的数据集百分比。
- `test_fraction_split`: 用于测试的数据集百分比（留存数据）。
- `validation_fraction_split`: 用于验证的数据集百分比。
- `budget_milli_node_hours`: （可选）以毫小时为单位指定的最大训练时间（1000 = 小时）。
- `disable_early_stopping`: 如果为`True`，服务可能会在使用完整预算之前完成训练，因为它相信无法进一步改进模型的目标测量。

完成`run`方法后，将返回`Model`资源。

训练管道的执行将需要最多20分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="salads_" + UUID,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    budget_milli_node_hours=20000,
    disable_early_stopping=False,
)

示例输出：

    信息：google.cloud.aiplatform.training_jobs:查看培训：
    https://console.cloud.google.com/ai/platform/locations/us-central1/training/2109316300865011712?project=759209241365
    信息：google.cloud.aiplatform.training_jobs:AutoMLImageTrainingJob项目/759209241365/locations/us-central1/trainingPipelines/2109316300865011712当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    信息：google.cloud.aiplatform.training_jobs:AutoMLImageTrainingJob项目/759209241365/locations/us-central1/trainingPipelines/2109316300865011712当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    信息：google.cloud.aiplatform.training_jobs:AutoMLImageTrainingJob项目/759209241365/locations/us-central1/trainingPipelines/2109316300865011712当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    信息：google.cloud.aiplatform.training_jobs:AutoMLImageTrainingJob项目/759209241365/locations/us-central1/trainingPipelines/2109316300865011712当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    信息：google.cloud.aiplatform.training_jobs:AutoMLImageTrainingJob项目/759209241365/locations/us-central1/trainingPipelines/2109316300865011712当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    ...
    信息：google.cloud.aiplatform.training_jobs:AutoMLImageTrainingJob运行完成。资源名称：projects/759209241365/locations/us-central1/trainingPipelines/2109316300865011712
    信息：google.cloud.aiplatform.training_jobs:模型可在projects/759209241365/locations/us-central1/models/1284590221056278528找到

## 评估模型

### [projects.locations.models.evaluations.list](https://cloud.devsite.corp.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.models.evaluations/list)

### [项目.位置.模型.评估列表](https://cloud.devsite.corp.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.models.evaluations/list)

## 查看模型评估分数
在您的模型训练完成后，您可以查看其评估分数。

首先，您需要获取对新模型的引用。与数据集一样，您可以使用部署模型时创建的模型变量的引用，或者列出项目中的所有模型。

In [ ]:
# Get model resource ID
models = aip.Model.list(filter="display_name=salads_" + UUID)

# Get a reference to the Model Service client
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
model_service_client = aip.gapic.ModelServiceClient(client_options=client_options)

model_evaluations = model_service_client.list_model_evaluations(
    parent=models[0].resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

*示例输出：*

    名称： "projects/759209241365/locations/us-central1/models/623915674158235648/evaluations/4280507618583117824"
    metrics_schema_uri： "gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml"
    指标 {
      结构值 {
        字段 {
          键： "auPrc"
          值 {
            number_value: 0.9891107
          }
        }
        字段 {
          键： "confidenceMetrics"
          值 {
            list_value {
              values {
                struct_value {
                  fields {
                    键： "precision"
                    值 {
                      number_value: 0.2
                    }
                  }
                  fields {
                    键： "recall"
                    值 {
                      number_value: 1.0
                    }
                  }
                }
              }

进行批量预测

### [predictions.batch-prediction](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions)

### [预测批量预测](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions)

获取测试项目

现在对您的Vertex模型进行批量预测。您将使用数据集中的任意示例作为测试项目。不要担心这些示例可能已经在训练模型时被使用 - 我们只是想演示如何进行预测。

In [ ]:
test_items = !gsutil cat $IMPORT_FILE | head -n2
cols_1 = str(test_items[0]).split(",")
cols_2 = str(test_items[1]).split(",")
if len(cols_1) == 11:
    test_item_1 = str(cols_1[1])
    test_label_1 = str(cols_1[2])
    test_item_2 = str(cols_2[1])
    test_label_2 = str(cols_2[2])
else:
    test_item_1 = str(cols_1[0])
    test_label_1 = str(cols_1[1])
    test_item_2 = str(cols_2[0])
    test_label_2 = str(cols_2[1])

print(test_item_1, test_label_1)
print(test_item_2, test_label_2)

### 复制测试项目

对于批量预测，请将测试项目复制到您的云存储桶中。

In [ ]:
file_1 = test_item_1.split("/")[-1]
file_2 = test_item_2.split("/")[-1]

! gsutil cp $test_item_1 $BUCKET_URI/$file_1
! gsutil cp $test_item_2 $BUCKET_URI/$file_2

test_item_1 = BUCKET_URI + "/" + file_1
test_item_2 = BUCKET_URI + "/" + file_2

### 制作批量输入文件

现在制作一个批量输入文件，将其存储在您的本地云存储桶中。批量输入文件可以是CSV或JSONL格式。在本教程中，您将使用JSONL格式。对于JSONL文件，您会为每个数据项（实例）的每一行制作一个字典条目。字典包含键/值对：

- `content`：图像的云存储路径。
- `mime_type`：内容类型。在我们的例子中，它是一个`jpeg`文件。

例如：

{'content': '[your-bucket]/file1.jpg', 'mime_type': 'jpeg'}

In [ ]:
import json

import tensorflow as tf

gcs_input_uri = BUCKET_URI + "/test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    data = {"content": test_item_1, "mime_type": "image/jpeg"}
    f.write(json.dumps(data) + "\n")
    data = {"content": test_item_2, "mime_type": "image/jpeg"}
    f.write(json.dumps(data) + "\n")

print(gcs_input_uri)
! gsutil cat $gcs_input_uri

### 发送批量预测请求

现在您的模型资源已经训练好了，您可以通过调用batch_predict()方法来进行批量预测，需要传入以下参数：

- `job_display_name`: 批量预测作业的人类可读名称。
- `gcs_source`: 一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`: 用于存储批量预测结果的云存储位置。
- `sync`: 如果设置为True，则调用将会阻塞，等待异步批量作业完成。

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="salads_" + UUID,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    sync=False,
)

print(batch_predict_job)

示例输出：

    INFO: google.cloud.aiplatform.jobs: 创建批量预测作业
    <google.cloud.aiplatform.jobs.BatchPredictionJob object at 0x7f806a6112d0>正在等待上游依赖项完成。
    INFO: google.cloud.aiplatform.jobs: 已创建BatchPredictionJob。资源名称：projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296
    INFO: google.cloud.aiplatform.jobs: 要在另一个会话中使用此BatchPredictionJob：
    INFO: google.cloud.aiplatform.jobs: bpj = aiplatform.BatchPredictionJob('projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296')
    INFO: google.cloud.aiplatform.jobs: 查看批预测作业：
    https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/5110965452507447296?project=759209241365
    INFO: google.cloud.aiplatform.jobs: BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296当前状态：
    JobState.JOB_STATE_RUNNING

### 等待批处理预测任务完成

接下来，等待批处理任务完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批处理预测任务完成。

In [ ]:
batch_predict_job.wait()

示例输出：

INFO: google.cloud.aiplatform.jobs: BatchPredictionJob已创建。 资源名称：projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328
INFO: google.cloud.aiplatform.jobs: 要在另一个会话中使用此BatchPredictionJob：
INFO: google.cloud.aiplatform.jobs: bpj = aiplatform.BatchPredictionJob('projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328')
INFO: google.cloud.aiplatform.jobs: 查看批量预测作业：
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/181835033978339328?project=759209241365
INFO: google.cloud.aiplatform.jobs: BatchPredictionJob项目/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328当前状态：
JobState.JOB_STATE_RUNNING
INFO: google.cloud.aiplatform.jobs: BatchPredictionJob项目/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328当前状态：
JobState.JOB_STATE_RUNNING
INFO: google.cloud.aiplatform.jobs: BatchPredictionJob项目/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328当前状态：
JobState.JOB_STATE_RUNNING
INFO: google.cloud.aiplatform.jobs: BatchPredictionJob项目/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328当前状态：
JobState.JOB_STATE_RUNNING
INFO: google.cloud.aiplatform.jobs: BatchPredictionJob项目/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328当前状态：
JobState.JOB_STATE_RUNNING
INFO: google.cloud.aiplatform.jobs: BatchPredictionJob项目/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328当前状态：
JobState.JOB_STATE_RUNNING
INFO: google.cloud.aiplatform.jobs: BatchPredictionJob项目/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328当前状态：
JobState.JOB_STATE_RUNNING
INFO: google.cloud.aiplatform.jobs: BatchPredictionJob项目/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328当前状态：
JobState.JOB_STATE_RUNNING
INFO: google.cloud.aiplatform.jobs: BatchPredictionJob项目/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328当前状态：
JobState.JOB_STATE_SUCCEEDED
INFO: google.cloud.aiplatform.jobs: BatchPredictionJob运行已完成。 资源名称：projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328

### 获取预测结果

接下来，从已完成的批量预测作业中获取结果。

结果将被写入你在批量预测请求中指定的云存储输出桶中。你可以调用iter_outputs()方法来获取包含结果的每个云存储文件的列表。每个文件以JSON格式包含一个或多个预测请求：

- `content`: 预测请求。
- `prediction`: 预测响应。
   - `ids`: 每个预测请求的内部分配的唯一标识符。
   - `displayNames`: 每个类别标签的类名。
   - `bboxes`: 每个检测到的对象的边界框。

In [ ]:
import json

import tensorflow as tf

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

示例输出：

    预测(predictions=[{'ids': ['225634079670796288', '2099131524656922624', '4404974533870616576', '6710817543084310528', '225634079670796288', '2099131524656922624', '6710817543084310528', '2099131524656922624', '225634079670796288', '671081754308431052…'}, 部署的模型ID='2225068486990757888'，解释=None）

做在线预测

### [predictions.deploy-model-api](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)

### [预测部署模型 API](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)

部署模型

接下来，部署您的模型进行在线预测。要部署模型，您需要调用`deploy`方法。

In [ ]:
endpoint = model.deploy()

*示例输出：*

    INFO:google.cloud.aiplatform.models:创建端点
    INFO:google.cloud.aiplatform.models:创建端点支持的LRO：projects/759209241365/locations/us-central1/endpoints/4867177336350441472/operations/4087251132693348352
    INFO:google.cloud.aiplatform.models:端点已创建。资源名称：projects/759209241365/locations/us-central1/endpoints/4867177336350441472
    INFO:google.cloud.aiplatform.models:要在另一个会话中使用此端点：
    INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/759209241365/locations/us-central1/endpoints/4867177336350441472')
    INFO:google.cloud.aiplatform.models:将模型部署到端点：projects/759209241365/locations/us-central1/endpoints/4867177336350441472
    INFO:google.cloud.aiplatform.models:部署端点模型支持的LRO：projects/759209241365/locations/us-central1/endpoints/4867177336350441472/operations/1691336130932244480
    INFO:google.cloud.aiplatform.models:端点模型已部署。资源名称：projects/759209241365/locations/us-central1/endpoints/4867177336350441472

### [predictions.online-prediction-automl](https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-automl)

### [predictions.在线预测-自动ML](https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-automl)

获取测试项

您将使用数据集中的任意示例作为测试项。不必担心这个示例很可能在训练模型时使用过 - 我们只是想演示如何进行预测。

In [ ]:
test_items = !gsutil cat $IMPORT_FILE | head -n1
cols = str(test_items[0]).split(",")
if len(cols) == 11:
    test_item = str(cols[1])
    test_label = str(cols[2])
else:
    test_item = str(cols[0])
    test_label = str(cols[1])

print(test_item, test_label)

### 进行预测

现在您的 `Model` 资源已部署到一个 `Endpoint` 资源中，您可以通过向 Endpoint 资源发送预测请求来进行在线预测。

#### 请求

由于在这个例子中您的测试项目位于一个 Cloud 存储桶中，您需要使用 `tf.io.gfile.GFile()` 来打开并读取图像的内容。为了将测试数据传递给预测服务，您需要将字节编码为 base64 格式 -- 这样可以确保在网络上传输二进制数据时内容不会被修改。

每个实例的格式如下：

    { 'content': { 'b64': base64编码的字节 } }

由于 `predict()` 方法可以接受多个项目（实例），请将您的单个测试项目作为一个测试项目的列表发送。

#### 响应

从 `predict()` 调用中得到的响应是一个 Python 字典，具有以下条目：

- `ids`: 每个预测请求的内部分配的唯一标识符。
- `displayNames`: 每个类别标签的类别名称。
- `confidences`: 每个类别标签的预测置信度，介于 0 和 1 之间。
- `bboxes`: 每个检测到对象的边界框。
- `deployed_model_id`: 执行预测的部署的 Model 资源的 Vertex AI 标识符。

In [ ]:
import base64

import tensorflow as tf

with tf.io.gfile.GFile(test_item, "rb") as f:
    content = f.read()

# The format of each instance should conform to the deployed model's prediction input schema.
instances = [{"content": base64.b64encode(content).decode("utf-8")}]

prediction = endpoint.predict(instances=instances)

print(prediction)

*Example output:*

预测（predictions = [{'ids'：['2250776168359788544'，'1097854663752941568'，'2250776168359788544'，'1097854663752941568'，'1097854663752941568'，'2250776168359788544'，'5709540682180329472'，'1097854663752941568'，'2250776168359788544'，'2250776168359788544'，'3403697672966635520'，'2250776168359788544'，'3403697672966635520'，'5709540682180329472'，'8015383691394023424'，'5709540682180329472'，'3403697672966635520'，'3403697672966635520'，'3403697672966635520'，'1097854663752941568'，'2250776168359788544'，'2250776168359788544'，'1097854663752941568'，'2250776168359788544'，'2250776168359788544'，'3403697672966635520'，'8015383691394023424'，'1097854663752941568'，'2250776168359788544'，'1097854663752941568'，'2250776168359788544'，'2250776168359788544'，'5709540682180329472'，'1097854663752941568'，'1097854663752941568'，'2250776168359788544'，'5709540682180329472'，'8015383691394023424'，'2250776168359788544'，'3403697672966635520' ]，'displayNames'：['沙拉'，'烘焙食品'，'沙拉'，'烘焙食品'，'烘焙食品'，'沙拉'，'海鲜'，'烘焙食品'，'沙拉'，'沙拉'，'奶酪'，'沙拉'，'奶酪'，'海鲜'，'西红柿'，'海鲜'，'奶酪'，'奶酪'，'奶酪'，'烘焙食品'，'沙拉'，'沙拉'，'烘焙食品'，'沙拉'，'沙拉'，'奶酪'，'西红柿'，'烘焙食品'，'沙拉'，'烘焙食品'，'沙拉'，'沙...
嵘綻= [[0.423218071,0.979368508,0.339486301,0.953063667]，[0.00295934081,0.562559605,0.0866475701,0.733601213]，[0.446993053,0.985957086,0.103116274,1.0]，[0.0,0.798019052,0.0843312368,0.910729289]，[0.200755239,0.878952861,0.161852047,0.962031841]，[0.141423374,0.893875718,0.0966037139,0.975178]，[0.446385473,0.954654098,0.322997868,0.955895185]，[0.0540368557,0.735850155,0.22394672,0.783547163]，[0.0511586964,0.590276837,0.247575879,0.904480815]，[0.291286081,0.943224788,0.742888927,0.981343627]，[0.0209743679,0.534335613,0.145708829,0.751705289]，[0.286338538,0.956577778,0.584680378,0.96791]，[0.493991911,0.92484349,0.385724247,0.975459754]，[0.320449412,0.9483518,0.16373083,1.0]，[0.458497226,0.937700331,0.342862546,0.935759783]，[0.0386219025,0.711036801,0.014...
确信=[0.99927026，0.998335898，0.122888528，0.0565881766，0.0334013812，0.0227512382，0.019394394，0.0163640734，0.0106114028，0.0095577，0.0092699714，0.00591981411，0.00551535888，0.00492637418，0.00376707385，0.00370016089，0.00318370084，0.00264257635，0.00248732581，0....
练好的模型ID ='5770527293638180864'，解释=None）

取消部署模型

当您完成预测后，您可以从“Endpoint”资源中取消部署模型。这将取消所有计算资源并停止对部署模型的计费。

In [ ]:
endpoint.undeploy_all()

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
import os

delete_bucket = False

# Delete the dataset using the Vertex dataset object

dataset.delete()

# Delete the model using the Vertex model object
model.delete()

# Delete the endpoint using the Vertex endpoint object
endpoint.delete()

# Delete the AutoML or Pipeline trainig job

dag.delete()

# Delete the batch prediction job using the Vertex batch prediction object
batch_predict_job.delete()

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI